This notebook is meant to be a simpler version of this [official doc](https://docs.modular.com/mojo/manual/basics/#syntax-and-semantics)
created during going over the official doc

# Mojo
is a designed to become a superset of Python, for example you can run top-level code just like in Python

In [2]:
print("Hello Mojo!")

Hello Mojo!


# Language basics
- Mojo is a compiled language
- Mojo code can be ahead-of-time (AOT) or just-in-time (JIT) compiled.

Mojo programs require a main() function as the entry point to the program \
*Of course, if you’re building a Mojo module (an API library), not a Mojo program, then your file doesn’t need a main() function (because it will be imported by other programs that do have one).*

In [3]:
fn main():
    var x: Int = 1
    x += 1
    print(x)
main()

2


As u can see, we used `fn main()` instead of `def main()` that you're familiar from python. Both actually work in Mojo

In [4]:
def main():
    x = 1
    x += 1
    print(x)
main()

2


### So why would you use `fn main()`? 

- `fn` is a ** Function ** 
- `fn` enforces strongly-type and memory-safe behaviors, while `def` provides more dynamic behaviors

## Variables
You can declare variables
- `var` creates mutable value
- `let` creates an immutable value

If we were to change `var` to `let` in `fn main()`, we would get a compiler error.

```Mojo
fn main():
    let x: Int = 1
    x += 1
    print(x)

error: Expression [15]:3:5: expression must be mutable for in-place operator destination
    x += 1
    ^
```

The compiler returns an error because `let` variable is immutable, so we can't increment it.
If we were to delete `var` completely, we would also get an error, because `fn` require explicit variable declarations.

```Mojo
fn main():
    x = 1
    x += 1
    print(x)

error: Expression [18]:2:5: use of unknown declaration 'x', 'fn' declarations require explicit variable declarations
    x = 1
    ^
```

Finally, notice that the `x` variable has an explicit `Int` type. Declaring the type is **not required** for variables in `fn`.

In [5]:
fn do_math():
    let x: Int = 1
    let y = 2
    print(x + y)

do_math()

3


## Function arguments and returns

Although types aren't required for variables declared in the function body, they are required for arguments and returns values for an `fn` function.

As seen in the below example:

In [6]:
fn add(x: Int, y: Int) -> Int:
    return x + y

add(3,4) # no __rep__? might be added in later version.

In python I could do 
```python
def add(x,y):
    return x+y
add(3,4)
```
jupyter output:    7

In [7]:
print(add(3,4))

7


## Optional arguments and keyword arguments

As in python, u can specify default values to the arguments
As seen in the below example:

In [8]:
fn pow(base: Int, exp: Int = 2) -> Int:
    return base ** exp

# Using default value for `exp`
print(pow(3))

# Using keyword argument names (with order reversed)
print(pow(exp=3,base=2))

9
8


**Note**: Mojo currently includes only partial support for keyword arguments, so some features such as keyword-only arguments and variadic keyword arguments (e.g. `**kwargs`) are not supported yet, as of `21.09.2023`.

## Argument mutability and ownership

Mojo supports full value semantics and enforces memory safety with a robust value ownership model.

Bellow function, `add()` doesn't modify `x` or `y`, it only reads the values. In fact, as written, the function *cannot* modify them because `fn` arguments are **immutable references**, by default.

It is called "borrowing" and although it's the default for `fn` function, you can make it explicit with `borrowed` declaration.

In [9]:
fn add(borrowed x: Int, borrowed y: Int) -> Int:
    return x+y

var x = 1
var y = 2
print(add(x, y))

3


NOTE:
Formatting such as `f"strings"` `.2f` are not supported as of `21.09.2023`

If we want the arguments to be mutable, we need to declare the argument convention as `inout`. This means that changes to the variables **inside** a function are visible **outside** the function.


In [10]:
fn add_inout(inout x:Int, inout y: Int) -> Int:
    x += 1
    y += 1
    return x + y

var x = 1
var y = 2
z = add_inout(x,y)
print(x,y,z)

2 3 5


Another way is to declare the argument as `owned`, provides full ownership of the value (it's mutable and guaranteed unique).

In [11]:
fn set_fire(owned text: String) -> String:
    text += "🔥"
    return text

fn mojo():
    let a: String = "mojo"
    let b = set_fire(a)
    print(a)
    print(b)

mojo()

mojo
mojo🔥


In this case, Mojo makes a copy of `a` string and passes it as the `text` argument. The original `a` string is still alive and well.

However, if you don't want to make a copy, then you can add the `^` "transfer" operator when you pass `a` to the function. \
The transfer operator effectively destroys the local variable, any attempt to call upon it later causes a compiler error.

In [12]:
fn mojo():
    let a: String = "mojo"
    let b = set_fire(a^)
    # print(a) will result in a error    
    print(b)
mojo()

mojo🔥


# Structures

A `struct` in Mojo is similar to a `class` in Python. However, Mojo structs are completely static -- they are bound at compile-time, so they do not allow dynamic dispatch or any runtime changes to the structure.

NOTE:
Mojo will also support classes in the future

Example of basic struct:

In [13]:
struct MyPair:
    var first: Int
    var second: Int

    fn __init__(inout self,first: Int, second: Int):
        self.first  = first
        self.second =  second

    fn dump(self):
        print(self.first, self.second)


let mine = MyPair(2, 4)
mine.dump()

2 4


## Python integration

Although Mojo is still a work in progress and is not a full superset of Python yet, Mojo has a built-in mechanism to import Python modules as-is. \
For example, here's how you can import and use NumPy.

In [14]:
from python import Python

let np = Python.import_module("numpy")

ar = np.arange(15).reshape(3,5)
print(ar)
print(ar.shape)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]
(3, 5)
